# ``orsopy`` Integration

``python-RAT`` contains some integration with ``orsopy``, allowing for convenient interaction with the ``.ort`` file format. This integration is available through the `RATapi.utils.orso` submodule.

In [ ]:
import RATapi.utils.orso

## Creating models from the ORSO model description language

The [ORSO model description format](https://www.reflectometry.org/advanced_and_expert_level/file_format/simple_model) allows the description of a standard slab model as a one-line string, provided that all the layer materials are defined [in the ORSO SLD database](https://slddb.esss.dk/slddb/).

The function `RATapi.utils.orso.orso_model_to_rat` function can read a model and return an `ORSOSample` dataclass, which gives bulk in and bulk out parameters for the model, a list of all layers defined in the model, and all the parameters needed to define those layers as RAT models. 

**Note:** the ORSO format gives the thicknesses of materials in *nanometres*. When we convert them to RAT parameters, the units will be converted to Angstroms.

For example, the string `air | Ni 100 | SiO2 0.5 | Si` describes a 1000 angstrom nickel film backed by a 5 angstrom silicon oxide layer. The bulk-in and bulk-out are air and silicon respectively. The roughnesses and SLDs will be calculated or taken from the ORSO SLD database.

In [ ]:
# create the RAT parameters and layers from this model
sample = RATapi.utils.orso.orso_model_to_rat("air | Ni 100 | SiO2 0.5 | Si")
print(sample)

You can also set `absorption=True` and the model will account for absorption. For example if we change the nickel film for a boron carbide film and want to account for its relatively high absorption, we can add it to the output:

In [ ]:
sample = RATapi.utils.orso.orso_model_to_rat("vacuum | B4C 100 | SiO2 0.5 | Si", absorption=True)
print(sample)

Finally, ORSO supports defining repeated layers using parentheses. For example, if we had a polarising multilayer of 5 repetitions of 70 angstrom silicon and 70 angstrom iron, we could represent it as `air | 5 ( Si 7 | Fe 7 ) | Si`.

RAT will only create the number of layers and parameters necessary, but the `ORSOSample` object's `model` attribute will give a list of layer names with the structure of the model preserved, which can be given as the layer model for a Contrast.

In [ ]:
sample = RATapi.utils.orso.orso_model_to_rat("air | 5 ( Si 7 | Fe 7 ) | Si")
print(sample)

## Reading in data and models from .ort files

RAT can also load both data and model information from an .ort file. This is done through the `ORSOProject` object, which takes a file path and can also optionally account for absorption.

The example data file we use here is example data for an unknown film on deposited on silicon.

In [ ]:
import pathlib
data_path = pathlib.Path("../data")

orso_data = RATapi.utils.orso.ORSOProject(data_path / "c_PLP0011859_q.ort")
print(orso_data)

The `ORSOProject` object contains two lists: `ORSOProject.data` and `ORSOProject.samples`. The former is a list of Data objects with each dataset defined in the file, and the latter is a list of `ORSOSample` objects (like above) with model information. Note that if the .ort file does not define a model for a dataset, that index of `ORSOProject.samples` will be None.

It's then easy to access this data to create a RAT `Project` that represents our data.

In [ ]:
from RATapi.models import Background, Contrast, Parameter, Resolution

dataset = orso_data.data[0]
sample = orso_data.samples[0]

project = RATapi.Project(
    name = "Example Project",
    geometry = "substrate/liquid",
    parameters = sample.parameters,
    bulk_in = [sample.bulk_in],
    bulk_out = [sample.bulk_out],
    scalefactors = [Parameter(name="Scalefactor", min=0, value=0.34, max=1.5)],
    background_parameters = [Parameter(name="Background Parameter", min=0, value=2e-6, max=1)],
    backgrounds = [Background(name="Background", type="constant", source="Background Parameter")],
    resolutions = [Resolution(name="Data Resolution", type="data")],
    data = [dataset],
    layers = sample.layers,
    contrasts = [Contrast(
        name = "prist4",
        data = dataset.name,
        background = "Background",
        bulk_in = sample.bulk_in.name,
        bulk_out = sample.bulk_out.name,
        scalefactor = "Scalefactor",
        resolution = "Data Resolution",
        model = sample.model,
    )]
)

controls = RATapi.Controls()
project, results = RATapi.run(project, controls)
RATapi.plotting.plot_ref_sld(project, results)